# Message Style

*Characteristics:*
- Collaboration Style: **Events = Choreography**
- System Style: **Reactive**
- Communication Style: **Asynchronous**
- Flow: **-**

## Process / Scenario

The following "Order Fulfilment" process model depicts the choreography happening among the three microservices defined below.

![](/images/Choreography.png)

### Tasks
Instruction to run/do the exercise:
1. You may register on [CloudAMQP for a free RabbitMQ](https://www.cloudamqp.com) instance (optional).
2. Add the AMQP URL of your own RabbitMQ instance or a provided one to the follwoing textbox:

In [ ]:
amqp_url = ''

3. Make sure that you run the notebook with the right tenant:

In [ ]:
tenant = 'showcase'

4. Run the notebook in Deepnote.
5. Inspect the Deepnote output.

In [ ]:
if not amqp_url:
    import os
    amqp_url = os.environ.get('AMQP_URL')

## Message Model

Below you will find the message model that is used for the message exchange, including `businessKey` generation using a UUID and `tenant` configuration:

In [ ]:
from typing import Optional
from pydantic import BaseModel
import uuid


class Message(BaseModel):
    businessKey: str
    tenant: str
    command: Optional[str] = None
    event: Optional[str] = None
    payload: Optional[str] = None


message = Message(
    **{
        "businessKey": str(uuid.uuid1()),
        "tenant": tenant
    }
)

## Message Queues

In the following, the [Pika](https://pika.readthedocs.io) library providing AMQP protocol in Python is used to connect to RabbitMQ and declare three queues `order`, `payment`, and `inventory`:

In [ ]:
import pika

params = pika.URLParameters(amqp_url)
# Connect to RabbitMQ
connection = pika.BlockingConnection(params)
channel = connection.channel()
# Declare order queue
channel.queue_declare(queue="order")
# Declare payment queue
channel.queue_declare(queue="payment")
# Declare inventory queue
channel.queue_declare(queue="inventory")

connection.close()


## Initial Event

In the following, an initial event `Order_Placed` is published to mimic a checkout and initialize the choreography:

In [ ]:
import pika, pickle

params = pika.URLParameters(amqp_url)
connection = pika.BlockingConnection(params)
channel = connection.channel()

# Define and publish (send) the Order_Placed event
message.event = "Order_Placed"
channel.basic_publish(exchange="", routing_key="order", body=pickle.dumps(message))
print("Order_Placed message sent.")

connection.close()


## Message Subscription of Microservices

In the following, three microservices are imitated, each of which has a subscription of one queue. After a message is received, a subsequent event is queued again.

In [ ]:
import pika, pickle, signal

params = pika.URLParameters(amqp_url)
connection = pika.BlockingConnection(params)
channel = connection.channel()

# Payment microservice callback function
def payment(channel, method, properties, body):
    message = pickle.loads(body)
    print("Received " + str(message))
    message.event = "Payment_Done"
    channel.basic_publish(
        exchange="", routing_key="payment", body=pickle.dumps(message)
    )
    return


# Inventory microservice callback function
def inventory(channel, method, properties, body):
    message = pickle.loads(body)
    print("Received " + str(message))
    message.event = "Goods_Fetched"
    channel.basic_publish(
        exchange="", routing_key="inventory", body=pickle.dumps(message)
    )
    return


# Shipment microservice callback function
def shipment(channel, method, properties, body):
    message = pickle.loads(body)
    print("Received " + str(message))
    message.event = "Goods_Shipped"
    channel.basic_publish(
        exchange="", routing_key="shipment", body=pickle.dumps(message)
    )
    return


# Subscriptions on queues
channel.basic_consume("order", payment, auto_ack=True)
channel.basic_consume("payment", inventory, auto_ack=True)
channel.basic_consume("inventory", shipment, auto_ack=True)


def close_handler(signal, frame):
    connection.close()


# Run consuming server
try:
    signal.signal(signal.SIGINT, close_handler)
    channel.start_consuming()
except Exception as e:
    print(e)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ede9f4cd-6ff6-4814-b5e4-540400368401' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>